<a href="https://colab.research.google.com/github/Stellariser/-/blob/main/INT303_cw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task1: Data Scraping

In [ ]:
#@title Dependencies
!pip install requests
!pip install bs4
!pip install pandas
!pip install tqdm
!pip install seaborn
!pip install re
import requests
import bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
for i in range(1,51):
    print(base_url.format(i))

http://books.toscrape.com/catalogue/page-1.html
http://books.toscrape.com/catalogue/page-2.html
http://books.toscrape.com/catalogue/page-3.html
http://books.toscrape.com/catalogue/page-4.html
http://books.toscrape.com/catalogue/page-5.html
http://books.toscrape.com/catalogue/page-6.html
http://books.toscrape.com/catalogue/page-7.html
http://books.toscrape.com/catalogue/page-8.html
http://books.toscrape.com/catalogue/page-9.html
http://books.toscrape.com/catalogue/page-10.html
http://books.toscrape.com/catalogue/page-11.html
http://books.toscrape.com/catalogue/page-12.html
http://books.toscrape.com/catalogue/page-13.html
http://books.toscrape.com/catalogue/page-14.html
http://books.toscrape.com/catalogue/page-15.html
http://books.toscrape.com/catalogue/page-16.html
http://books.toscrape.com/catalogue/page-17.html
http://books.toscrape.com/catalogue/page-18.html
http://books.toscrape.com/catalogue/page-19.html
http://books.toscrape.com/catalogue/page-20.html
http://books.toscrape.com/cat

In [ ]:
#@title Attempts
result = requests.get('http://books.toscrape.com/catalogue/page-1.html')
soup = bs4.BeautifulSoup(result.text, 'lxml')
# soup
soup.select('.product_pod')
print(len(soup))

7


In [ ]:
#@title Start Scraping
titles = []
urls = []
base_book_path = 'https://books.toscrape.com/catalogue/{}'

for i in range (1,51):
    scrape_url = base_url.format(i)

    result = requests.get(scrape_url)
    soup = bs4.BeautifulSoup(result.text, 'lxml')
    books = soup.select('.product_pod')

    for book in books:
        url = base_book_path.format(book.select('a')[0]['href'])
        title = book.select('a')[1]['title']
        titles.append(title)
        urls.append(url)

In [ ]:
# titles
# urls
print(len(titles))
print(len(urls))

1000
1000


In [ ]:
import tqdm
import re
num_rev = []
rating = []
prices = []
Product_types = []
categories = []
availability = []
i = 0

dic = {"One":1,"Two":2,"Three":3,"Four":4,"Five":5,"Zero":0}

for idx, url in enumerate(tqdm.tqdm(urls)):
  result = requests.get(url)
  soup = bs4.BeautifulSoup(result.text, 'lxml')
  table = soup.select('td')
  links = soup.select('a')
  rating_prop = soup.select('p')[2]['class'][1]
  # print(rating_prop)
  rating.append(dic[rating_prop])
  # print(rating)
  category = links[3].contents[0]
  # print('cat = ', category)
  categories.append(category)
  rev = table[6].contents[0]
  # print('# rev = ', rev)
  num_rev.append(rev)
  ava = table[5].contents[0]
  # print('avails = ', ava)
  value = re.findall(r'\d+',ava)[0]
  # print(value)
  availability.append(value)
  price = table[2].contents[0][-5:]
  # print('price = ',price)
  prices.append(price)
  Product_type = table[1].contents[0]
  # print('type = ' ,Product_type)
  Product_types.append(Product_type)

 10%|█         | 104/1000 [01:21<11:39,  1.28it/s]

In [ ]:
#@title Checking Length
print(len(num_rev)) 
print(len(rating))
print(len(prices))
print(len(Product_types))
print(len(categories))
print(len(availability))

In [ ]:
#@title Save as CSV
import pandas as pd
general_list = [num_rev, rating, prices, Product_types, categories, availability]
cols=['reviews', 'rating', 'prices', 'type', 'categories', 'availability']
df = pd.DataFrame(columns=titles, index=cols, data=general_list)
df = df.transpose()
df.to_csv('./df.csv', encoding="utf_8_sig")

## Result Visualization

In [ ]:
df

# Task2: Exploratory Data Analysis

If a data scientist want to purchase a book, here are few steps one would do to figure out what's one's best choice. 

***BEHOLD!***


![8f69bbd2d539b60058e386e3fe50352ac75cb7da.gif](https://pic.jitudisk.com/public/2022/10/15/0429058e5828a.gif)

In [ ]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Basic Information

Before we start EDA, let's look into the basic information of the dataframe by default APIs. Firstly, by `df.info()`, we can check the statistics of the data structure and types.

In [ ]:
df.info()

Secondly, by `df.describe()` we can look into the 

In [ ]:
df.describe()

## Basic Distribution

With all the information above, we would like to discover the general distribution for all the meaningful numerical attributes. 

The visualizations of distributions of ***Prices, Ratings and In stock number (Availability)*** are are provided below.

In [ ]:
sns.distplot(df["rating"])

In [ ]:
sns.distplot(df["prices"])

In [ ]:
sns.distplot(df["availability"])

In [ ]:
cat_series = df['categories'].value_counts()
y_cor = list(cat_series.values) 
x_cor = list(np.arange(len(y_cor)))

f, ax = plt.subplots(figsize=(70, 30))
sns.barplot(y=cat_series.values, x=cat_series.index,orient='v', alpha=0.8, color='red')
for x,y in zip(x_cor,y_cor): 
    plt.text(x, y+1, '%s' % y, ha='center', va= 'bottom',fontsize=14)
plt.title('Categories distribution', size = 40)
plt.show()